<a href="https://colab.research.google.com/github/lkforward/flower/blob/master/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ls

Mounted at /content/gdrive
code  gdrive  sample_data


# Load the U-Net Model Implementation

In [5]:
# #Clone the code into google drive: 

# Codebase 1:
# !git clone https://github.com/vlievin/Unet.git /content/gdrive/My\ Drive/kaggle_cloud/code_myunet

# Codebase 2:
# !git clone https://github.com/lyakaap/Kaggle-Carvana-3rd-Place-Solution.git /content/gdrive/My\ Drive/kaggle_cloud/code_3rdplace_unet

Cloning into '/content/gdrive/My Drive/kaggle_cloud/code_3rdplace_unet'...
remote: Enumerating objects: 57, done.
remote: Total 57 (delta 0), reused 0 (delta 0), pack-reused 57
Unpacking objects: 100% (57/57), done.


In [6]:
# !ln -sfn /content/gdrive/My\ Drive/kaggle_cloud/code_myunet code
# !ln -sfn /content/gdrive/My\ Drive/kaggle_cloud/code_3rdplace_unet code
!ls code

losses.py	    model.py	      network.png  train.py
make_submission.py  model_pytorch.py  README.md


In [0]:
# #If you want to view the code from a pop-up window: 
# %pycat code/unet.py
# %pycat code/model_pytorch.py

In [0]:
# %%writefile code/model_pytorch.py

import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvActivation(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=1, dilation=1,
                 activation=nn.ReLU(inplace=True)):
        super(ConvActivation, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                              stride, padding, dilation)
        self.activation = activation

    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        return x


class ConvBNActivation(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=1, dilation=1,
                 activation=nn.ReLU(inplace=True)):
        super(ConvBNActivation, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                              stride, padding, dilation)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = activation

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x


class ConvBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=1, dilation=1,
                 batch_norm=False, activation=nn.ReLU(inplace=True)):
        super(ConvBlock, self).__init__()
        conv = ConvBNActivation if batch_norm else ConvActivation
        self.block = nn.Sequential(
            conv(in_channels, out_channels, kernel_size,
                 stride, padding, dilation, activation),
            conv(out_channels, out_channels, kernel_size,
                 stride, padding, dilation, activation)
        )

    def forward(self, x):
        out = self.block(x)
        return out


class UpBlockWithSkip(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=1, dilation=1, up_mode='deconv',
                 batch_norm=False, activation=nn.ReLU(inplace=True)):
        assert up_mode in ('deconv', 'biupconv', 'nnupconv')
        super(UpBlockWithSkip, self).__init__()

        if up_mode == 'deconv':
            self.up = nn.ConvTranspose2d(
                in_channels, out_channels,
                kernel_size=4, stride=2, padding=1)
        elif up_mode == 'biupconv':
            self.up = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2,
                            align_corners=False),
                nn.Conv2d(in_channels, out_channels, kernel_size=1)
            )
        elif up_mode == 'nnupconv':
            self.up = nn.Sequential(
                nn.Upsample(mode='nearest', scale_factor=2,
                            align_corners=False),
                nn.Conv2d(in_channels, out_channels, kernel_size=1)
            )

        self.conv_block = ConvBlock(
            out_channels * 2, out_channels, kernel_size,
            stride, padding, dilation, batch_norm, activation)

    def forward(self, x, bridge):
        up = self.up(x)
        out = torch.cat([up, bridge], 1)
        out = self.conv_block(out)

        return out


class DilatedUNet(nn.Module):
    def __init__(self, in_channels=3, classes=1, depth=3,
                 first_channels=44, padding=1,
                 bottleneck_depth=6, bottleneck_type='cascade',
                 batch_norm=False, up_mode='deconv',
                 activation=nn.ReLU(inplace=True)):

        assert bottleneck_type in ('cascade', 'parallel')
        super(DilatedUNet, self).__init__()

        self.depth = depth
        self.bottleneck_type = bottleneck_type

        conv = ConvBNActivation if batch_norm else ConvActivation

        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(
                ConvBlock(prev_channels, first_channels * 2**i, 3,
                          padding=padding, batch_norm=batch_norm,
                          activation=activation))
            prev_channels = first_channels * 2**i

        self.bottleneck_path = nn.ModuleList()
        for i in range(bottleneck_depth):
            bneck_in = prev_channels if i == 0 else prev_channels * 2
            self.bottleneck_path.append(
                conv(bneck_in, prev_channels * 2, 3,
                     dilation=2**i, padding=2**i, activation=activation))

        prev_channels *= 2

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth)):
            self.up_path.append(
                UpBlockWithSkip(prev_channels, first_channels * 2**i, 3,
                                up_mode=up_mode, padding=padding,
                                batch_norm=batch_norm,
                                activation=activation))
            prev_channels = first_channels * 2**i

        self.last = nn.Conv2d(prev_channels, classes, kernel_size=1)

    def forward(self, x):
        bridges = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            bridges.append(x)
            x = F.avg_pool2d(x, 2)

        dilated_layers = []
        for i, bneck in enumerate(self.bottleneck_path):
            if self.bottleneck_type == 'cascade':
                x = bneck(x)
                dilated_layers.append(x.unsqueeze(-1))
            elif self.bottleneck_type == 'parallel':
                dilated_layers.append(bneck(x.unsqueeze(-1)))
        x = torch.cat(dilated_layers, dim=-1)
        x = torch.sum(x, dim=-1)

        for i, up in enumerate(self.up_path):
            x = up(x, bridges[-i-1])

        return self.last(x)

In [17]:
# net = UNet(in_channels=3, out_channels=4, num_hidden_features=[64, 128, 256], 
#            num_dilated_convs=2, n_resblocks=2, 
#            dropout_min=0., dropout_max=0.2)

net = DilatedUNet(in_channels=3, classes=4, depth=5)

print(net)

DilatedUNet(
  (down_path): ModuleList(
    (0): ConvBlock(
      (block): Sequential(
        (0): ConvActivation(
          (conv): Conv2d(3, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): ReLU(inplace=True)
        )
        (1): ConvActivation(
          (conv): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): ReLU(inplace=True)
        )
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): ConvActivation(
          (conv): Conv2d(44, 88, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): ReLU(inplace=True)
        )
        (1): ConvActivation(
          (conv): Conv2d(88, 88, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): ReLU(inplace=True)
        )
      )
    )
    (2): ConvBlock(
      (block): Sequential(
        (0): ConvActivation(
          (conv): Conv2d(88, 176, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act

In [18]:
input = torch.randn(16, 3, 320, 320)
output = net(input)
print(output)

tensor([[[[ 0.0226,  0.0278,  0.0194,  ...,  0.0214,  0.0171,  0.0257],
          [ 0.0236,  0.0196,  0.0312,  ...,  0.0274,  0.0282,  0.0258],
          [ 0.0210,  0.0256,  0.0272,  ...,  0.0295,  0.0239,  0.0250],
          ...,
          [ 0.0208,  0.0270,  0.0110,  ...,  0.0273,  0.0222,  0.0279],
          [ 0.0267,  0.0278,  0.0314,  ...,  0.0371,  0.0219,  0.0300],
          [ 0.0270,  0.0299,  0.0247,  ...,  0.0242,  0.0230,  0.0287]],

         [[-0.1230, -0.1218, -0.1299,  ..., -0.1290, -0.1358, -0.1313],
          [-0.1323, -0.1300, -0.1306,  ..., -0.1283, -0.1337, -0.1271],
          [-0.1315, -0.1158, -0.1212,  ..., -0.1350, -0.1235, -0.1312],
          ...,
          [-0.1283, -0.1232, -0.1355,  ..., -0.1260, -0.1171, -0.1308],
          [-0.1209, -0.1230, -0.1258,  ..., -0.1160, -0.1266, -0.1274],
          [-0.1280, -0.1287, -0.1275,  ..., -0.1268, -0.1234, -0.1243]],

         [[ 0.0389,  0.0436,  0.0301,  ...,  0.0441,  0.0437,  0.0436],
          [ 0.0440,  0.0477,  

In [20]:
output.min()

tensor(-0.1624, grad_fn=<MinBackward1>)

In [21]:
output.max()

tensor(0.0825, grad_fn=<MaxBackward1>)

# Model Training

# Model Prediction